# Libraries and preparation

refs:
- https://github.com/langchain-ai/langgraph/blob/main/examples/rag/langgraph_rag_agent_llama3_local.ipynb
- https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/agent_supervisor.ipynb?ref=blog.langchain.dev
- https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/hierarchical_agent_teams.ipynb?ref=blog.langchain.dev

MAP:REDUCE: https://langchain-ai.github.io/langgraph/how-tos/map-reduce/

In [2]:
import subprocess
import threading

#istallazione di ollama
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
######################################################################## 100.0%#=#=#                                                                          
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
def start_ollama():
    t = threading.Thread(target=lambda: subprocess.run(["ollama", "serve"]),daemon=True)
    t.start()

In [4]:
def pull_model(local_llm):
    !ollama pull local_llm

In [5]:
def start_model(local_llm):        
    t2 = threading.Thread(target=lambda: subprocess.run(["ollama", "run", local_llm]),daemon=True)
    t2.start()

In [6]:
%%capture --no-stderr
%pip install -U scikit-learn==1.3 langchain-ai21 ragas langchain-pinecone langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python nomic[local] langchain-text-splitters

In [7]:
# Tracing and api-keys
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["TAVILY_API_KEY"] = "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"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["PINECONE_API_KEY"] = "94ef7896-1fae-44d3-b8d2-0bd6f5f664f5"
os.environ["AI21_API_KEY"] = "KlINkh5QKw3hG1b5Hr75YDO7TwGoQvzn"

Bias detection model:

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1

bias_model_tokenizer = AutoTokenizer.from_pretrained("d4data/bias-detection-model")
bias_model = AutoModelForSequenceClassification.from_pretrained("d4data/bias-detection-model",from_tf=True)

- https://shap.readthedocs.io/en/latest/example_notebooks/text_examples/text_entailment/Textual%20Entailment%20Explanation%20Demo.html
- https://huggingface.co/facebook/bart-large-mnli

Entailment model (BART):

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

device = 0 if torch.cuda.is_available() else -1

bart_model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli",device=device)
bart_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")

In [ ]:
def BART_prediction(premise,hypothesis):
    #print(f"Premise: {premise}")
    #print(f"Hypo: {hypothesis}")
    input_ids = bart_tokenizer.encode(premise, hypothesis, return_tensors="pt")
    logits = bart_model(input_ids)[0]
    probs = logits.softmax(dim=1)

    max_index = torch.argmax(probs).item()

    bart_label_map = {0: "contradiction", 1: "neutral", 2: "entailment"}
    return bart_label_map[max_index]

# Tools

refs:
- https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/

In [ ]:
### Search
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_text_splitters import RecursiveCharacterTextSplitter

web_search_tool = TavilySearchResults(k=2)

# Indexing

Organizing external sources for the llm. Phase of indexing and chunking of docs refs:
- https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/
- https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/
- https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/
- Nomic embeddings: https://docs.nomic.ai/atlas/capabilities/embeddings#selecting-a-device

osservazione: si possono controllare gli indici direttamente da https://app.pinecone.io/organizations/-O2Tiw_0VD7HTOASPJE5/projects/2a95c518-e514-4d39-bed8-4b12fd90ad44/indexes

osservazione sul chuncking: https://dev.to/peterabel/what-chunk-size-and-chunk-overlap-should-you-use-4338

In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain_ai21 import AI21Embeddings

def create_retriever(index_name,top_k):
    vectorstore = PineconeVectorStore(
        index_name=index_name,
        embedding=AI21Embeddings(device="cuda")
    )
    return vectorstore.as_retriever(search_kwargs={"k": top_k})

def create_KBT_retrievers(aspects,top_k):
    retrievers = []
    for aspect in aspects:
        retriever = create_retriever(f"{aspect.lower()}-kbt",top_k)
        retrievers.append(retriever)
    return retrievers

# Query rewriting (multi-aspects)

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

def query_rewriting(local_llm):
    prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You have to rewrite an original query
        into a search-query based on a specified aspect. NO PREAMBLE: return only the rewriten-query.
        Here some examples of the task: \n
        
        Original query: "What about COVID19?"
        Aspect: "Health"
        Answer: "Symptoms of COVID19"
        \n ----- \n
        
        Original query: "COVID19 was fake?"
        Aspect: "Society"
        Answer: "What do people think about COVID19?"
        \n ----- \n
        
        Original query: "COVID19 was a hoax?"
        Aspect: "Technology"
        Answer: "Were there micro-chip into vaccines?"
        \n ----- \n
        
        <|eot_id|><|start_header_id|>user<|end_header_id|>
        Original query: {original_query}
        Aspect: {aspect}
        Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["aspect","original_query"],
    )
    llm = ChatOllama(model=local_llm, temperature=0) 
    query_generator = prompt | llm | StrOutputParser() 
    return query_generator

# Organizing outputs

## Without sections

In [ ]:
def final_answer(local_llm):
    prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> 
        You are an helpful assistant who must organize a coherent speech that comes from multiple sources. 
        Your job is to put these pieces of text together, maintaining correct punctuation and coherence in your speech. 
        Provide only the organized segments, without the preamble.
        <|eot_id|><|start_header_id|>user<|end_header_id|>
        Here are pieces of text: {answers}
        Result answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["answers"],
    )
    llm = ChatOllama(model=local_llm, temperature=0)
    final_answer = prompt | llm | StrOutputParser()
    return final_answer

#final_output = final_answer.invoke({"answers": answers})
#print(final_output)

## With section

In [ ]:
def final_answer_section(local_llm):
    prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> 
        You are an helpful assistant who must organize a coherent speech that comes from multiple sources. 
        Your job is to organize the text in section, you have some "aspects" and you have to organize 
        text 1 with aspect 1, text 2 with aspect 2 and so on. Provide only the organized segments, without the preamble.
        Here an example: \n \n
        
        Texts: [text1, text2, text3] \n
        Aspects: [aspect1, aspect2, aspect3] \n
        Answer: \n
        - aspect1 section: text1 \n
        - aspect2 section: text2 \n
        - aspect3 section: text3 \n
        
        <|eot_id|><|start_header_id|>user<|end_header_id|>
        Texts: {answers}
        Aspects: {aspects}
        Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["answers","aspects"],
    )
    llm = ChatOllama(model=local_llm, temperature=0)
    final_answer = prompt | llm | StrOutputParser()
    return final_answer

#final_output = final_answer.invoke({"answers": answers})
#print(final_output)

# Retrieval

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

#higher temperature more likely hallucinations

def retrieval_grader(local_llm):
    prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance
        of a retrieved document to a user question. If the document contains keywords related to the user question,
        grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
        Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
        Provide only the answer 'yes' or 'no', NOT ANYMORE. NO PREAMBLE. NO EXPLANATION.
        <|eot_id|><|start_header_id|>user<|end_header_id|>
        Here is the retrieved document: \n\n {document} \n\n
        Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
        """,
        input_variables=["question", "document"],
    )
    llm = ChatOllama(model=local_llm, temperature=0) #higher temperature more likely hallucinations
    retrieval_grader = prompt | llm | StrOutputParser()
    return retrieval_grader

# question = "agent memory"
# docs = retriever.invoke(question)
# doc_txt = docs[1].page_content
# print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

# Generating answer

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def rag_chain(local_llm):
    prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks.
        Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
        Use three sentences maximum and keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
        Question: {question}
        Context: {context}
        Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["question", "document"],
    )
    llm = ChatOllama(model=local_llm, temperature=0)
    # Chain
    rag_chain = prompt | llm | StrOutputParser()
    return rag_chain

# Run
# question = "agent memory"
# docs = retriever.invoke(question)
# generation = rag_chain.invoke({"context": docs, "question": question})
# print(generation)

# Hallucinations check (not used)

Per ora non uso hallucinations check

In [ ]:
def hallucination_grader(local_llm):
    prompt = PromptTemplate(
        template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether
        an answer is grounded in / supported by a set of facts. Give a binary score 'yes' or 'no' (both in lower case) to indicate
        whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a
        SINGLE KEY 'score' and NO preamble or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
        Here are the facts:
        \n ------- \n
        {documents}
        \n ------- \n
        Here is the answer: {generation}  <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["generation", "documents"],
    )
    llm = ChatOllama(model=local_llm, format="json", temperature=0)
    hallucination_grader = prompt | llm | JsonOutputParser()
    return hallucination_grader

#hallucination_grader.invoke({"documents": docs, "generation": generation})

# Answer check

In [ ]:
def answer_grader(local_llm):
    prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an
        answer is useful to resolve a question. Give a binary score 'yes' or 'no' (both in lower case) to indicate whether the answer is
        useful to resolve a question. Provide the binary score as a JSON with a SINGLE KEY 'score' and NO preamble or explanation.
         <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
        \n ------- \n
        {generation}
        \n ------- \n
        Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["generation", "question"],
    )
    llm = ChatOllama(model=local_llm, format="json", temperature=0)
    answer_grader = prompt | llm | JsonOutputParser()
    return answer_grader

#answer_grader.invoke({"question": question, "generation": generation})

# Routing (not used)

Per ora non applichiamo il routing!

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

def question_router(local_llm):
    prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an expert at routing a
        user question to a vectorstore or web search. Use the vectorstore for questions on LLM  agents,
        prompt engineering, and adversarial attacks. You do not need to be stringent with the keywords
        in the question related to these topics. Otherwise, use web-search. Give a binary choice 'web_search'
        or 'vectorstore' based on the question. Return the a JSON with a single key 'datasource' and
        no premable or explanation. Question to route: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["question"],
    )
    llm = ChatOllama(model=local_llm, format="json", temperature=0)
    question_router = prompt | llm | JsonOutputParser()
    return question_router
    
# question = "llm agent memory"
# docs = retriever.get_relevant_documents(question)
# doc_txt = docs[1].page_content
# print(question_router.invoke({"question": question}))

# Entailment

In [ ]:
def entailment_checker(local_llm):
    prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You have to perform a task of text-entailment.
        In particular, you must determine the relationship between a premise and a hypothesis. 
        "The possible relationships are:\n"
        "1. contradiction: The premise contradicts the hypothesis.\n"
        "2. entailment: The premise implies the hypothesis.\n"
        "3. neutral: The premise and hypothesis are neutral.\n\n"
        Premise and hypothesis are in 'contradiction', if in the hypothesis there are contradicting statements with the premise statements.
        There is an 'entailment' between premise and hypothesis if it is likely that from the premise you could deduce what is stated in the hypothesis. 
        Premise and hypothesis are 'neutral' if they are not in contradiction or in entailment.
        Provide only the answer with the category, NOT ANYMORE. NO PREAMBLE. NO EXPLANATION. \n
        Here some examples: \n
        
        "Example 1:\n"
        "Premise: COVID-19 is caused by the SARS-CoV-2 virus.\n"
        "Hypothesis: The SARS-CoV-2 virus causes COVID-19.\n"
        "Answer: Entailment\n\n"
    
        "Example 2:\\n"
        "Premise: Health agencies have stated that COVID-19 vaccines do not contain any microchips or tracking devices.\\n"
        "Hypothesis: COVID-19 vaccines contain microchips to monitor people..\\n"
        "Answer: Contradiction\\n\\n"

        "Example 3:\\n"
        "Premise: Social distancing can reduce the spread of COVID-19.\\n"
        "Hypothesis: Washing hands frequently is essential to prevent the spread of COVID-19.\\n"
        "Answer: Neutral\\n\\n"

        "Example 4:\\n"
        "Premise: COVID-19 has led to global economic disruptions.\\n"
        "Hypothesis: Many businesses have suffered financial losses due to the pandemic.\\n"
        "Answer: Entailment\\n\\n"

        "Example 5:\\n"
        "Premise: Wearing masks can help reduce the transmission of COVID-19.\\n"
        "Hypothesis: Masks are the only measure needed to prevent COVID-19.\\n"
        "Answer: Contradiction\\n\\n"

        "Example 6:\\n"
        "Premise: The COVID-19 pandemic has accelerated the development of remote work technologies.\\n"
        "Hypothesis: More people are working from home due to advancements in technology.\\n"
        "Answer: Neutral\\n\\n"
        
        "Example 7:\n"
        "Premise: Research has shown that COVID-19 primarily affects the respiratory system, but can also lead to complications in other organs.\n"
        "Hypothesis: COVID-19 affects only the respiratory system.\n"
        "Answer: Contradiction\n\n"
        
        "Example 8:\n"
        "Premise: Despite initial vaccine hesitancy, widespread vaccination campaigns have significantly reduced the rate of severe COVID-19 cases globally.\n"
        "Hypothesis: Vaccine hesitancy has prevented the reduction of severe COVID-19 cases.\n"
        "Answer: Contradiction\n\n"
        
        "Example 9:\n"
        "Premise: The COVID-19 pandemic has led to an unprecedented increase in remote work, with companies around the world adopting digital tools to maintain operations.\n"
        "Hypothesis: The pandemic has forced many companies to adapt to remote work.\n"
        "Answer: Entailment\n\n"
        
        "Example 10:\n"
        "Premise: Although vaccines have been effective in preventing severe disease, the emergence of new variants has led to concerns about vaccine efficacy.\n"
        "Hypothesis: Vaccines are completely ineffective against new variants.\n"
        "Answer: Contradiction\n\n"
        
        "Example 11:\n"
        "Premise: Governments worldwide have implemented various public health measures, including lockdowns, mask mandates, and social distancing, to control the spread of COVID-19.\n"
        "Hypothesis: Lockdowns were the only measure used to control the spread of COVID-19.\n"
        "Answer: Contradiction\n\n"
        
        "Example 12:\n"
        "Premise: The economic impact of COVID-19 has been severe, with a sharp increase in unemployment rates, especially in sectors such as hospitality and tourism.\n"
        "Hypothesis: The pandemic has disproportionately affected certain industries, leading to higher unemployment rates in those sectors.\n"
        "Answer: Entailment\n\n"
        
         <|eot_id|><|start_header_id|>user<|end_header_id|> Premise:
        {first_doc}
        \n ------- \n
        Hypothesis:
        \n ------- \n
        {second_doc}
        \n ------- \n
        Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["first_doc", "second_doc"],
    )
    llm = ChatOllama(model=local_llm, temperature=0)
    entailment_checker = prompt | llm | StrOutputParser()
    return entailment_checker

# Debiasing

https://learnprompting.org/docs/reliability/debiasing

In [ ]:
def debiasing_answer(local_llm):
    prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> The user will pass you text cointaining biases.
        Your role is to give a debiased version of the text. Give ONLY the debiased text, NO PREAMBLE, NO EXPLANATIONS.
        Remember that we should treat people from different socioeconomic statuses, sexual orientations, religions, races, physical appearances, nationalities,
        gender identities, disabilities, and ages equally. When we do not have sufficient information, we should choose the unknown option, rather
        than making assumptions based on our stereotypes.
         <|eot_id|><|start_header_id|>user<|end_header_id|> 
        {text}
        <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["text"],
    )
    llm = ChatOllama(model=local_llm, temperature=0)
    debiasing = prompt | llm | StrOutputParser()
    return debiasing

# Bias detection

In [62]:
# response: biased or non-biased

def bias_det_llm(local_llm):
    prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> 
        You will be provided with a text, and your task is to classify it into 
        one of two categories: "biased" or "non-biased." A text is considered "biased" 
        if it expresses a personal opinion, uses emotional or partial language, or promotes 
        a particular perspective at the expense of others. A text is considered "non-biased" 
        if it presents facts in a neutral and objective manner, without evident influences.
        Give ONLY the class "biased" or "non-biased", NO PREAMBLE, NO EXPLANATIONS.
        Below, I will show you some classification examples to help you understand 
        how to distinguish between the two types of texts. \n

        Example 1: \n
        Text: "The new educational program is a complete failure, designed with no regard for the needs of the students." \n
        Answer: biased \n

        Example 2: \n
        Text: "The new educational program was introduced with the goal of improving student performance in scientific subjects." \n
        Answer: non-biased \n

        Example 3: \n
        Text: "The current government has done a terrible job managing the economic crisis, causing more harm than good." \n
        Answer: biased \n

        Example 4: \n
        Text: "The government has implemented a series of measures to address the economic crisis, with mixed results according to various analysts." \n
        Answer: non-biased \n
   
         <|eot_id|><|start_header_id|>user<|end_header_id|> 
        Text: {text} \n
        Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["text"],
    )
    llm = ChatOllama(model=local_llm, temperature=0)
    bias_det_chain = prompt | llm | StrOutputParser()
    return bias_det_chain

# Hate speech detection

https://arxiv.org/html/2401.03346v1/#S4

In [ ]:
def hate_speech_detection(local_llm):
    prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an helpful assistant who has to detect the presence of hate speech.
            Hate speech is speech that attacks a person or group based on attributes such as race, religion, ethnic origin, national origin, sex, disability, sexual orientation, or gender identity. 
            You have to answer "yes" if it contains hate speech, or "no" if it doesn't contain hate speech. NO PREAMBLE, NO EXPLANATIONS.
            <|eot_id|><|start_header_id|>user<|end_header_id|> 
            Do you think this document contain hate speech? document: {document}.
            <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["document"],
    )
    llm = ChatOllama(model=local_llm, temperature=0)
    hate_speech_detection = prompt | llm | StrOutputParser()
    return hate_speech_detection

# Aspect agents

refs
- https://www.langchain.com/langgraph

In [88]:
from pprint import pprint
from typing import List, Annotated
import operator
import functools
import sklearn.metrics
import numpy as np

from langchain_core.documents import Document
from typing_extensions import TypedDict

from langgraph.graph import END, StateGraph, START

### State
class GraphState(TypedDict):
    """
    Represents the state of graph of aspect agents.
    """
    
    original_query: str
    query: str
    aspect: str
    aspect_id: int
    answers_agent: Annotated[List[str], operator.add]
    ord_aspects: Annotated[List[str], operator.add]
    my_answer: str
    web_search: str
    documents: List[str]
    documents_kbt: List[str]
        
def rewrite_query(state,verbose,llm):
    if verbose: 
        print("---REWRITING QUERY---")
        print(f"State: {state}")
    original_query = state["original_query"]
    aspect = state["aspect"]

    generation = query_rewriting(llm).invoke({"original_query": original_query, "aspect": aspect})
    #print(list(generation.values())) #Debug
    return {"query": generation}


def retrieve(state,verbose,retriever,retrievers_KBT):
    """
    Retrieve documents from vectorstore and from KBT

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    if verbose: 
        print("---RETRIEVE---")
        print(f"State: {state}")
        
    query = state["query"]
    aspect_id = state["aspect_id"]

    # Retrieval
    documents = retriever.invoke(query)
    documents_kbt = retrievers_KBT[aspect_id].invoke(query)
    
    #pprint(f"Documents retrieved: {documents}")
    #pprint(f"Documents KBT retrieved: {documents_kbt}")
    
    return {"documents": documents, "documents_kbt": documents_kbt, "query": query}


def generate(state,verbose,llm,fairness):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    if verbose:
        print("---GENERATE---")
        print(f"State: {state}")
    query = state["query"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain(llm).invoke({"context": documents, "question": query})
    aspect_id = state["aspect_id"]
    #print(f"Aspect agent {aspect_id} generates: {generation}") #Debug
    if fairness:
        return {"documents": documents, "query": query, "my_answer": generation}
    return {"documents": documents, "query": query, "my_answer": generation, "answers_agent": [generation], "ord_aspects": [state["aspect"]]}

def confirm_answer(state,verbose):
    """
    Confirm answer if there is no bias in the text.

    Args:
        state (dict): The current graph state

    """
    if verbose:
        print("---CONFIRM ANSWER---")
        print(f"State: {state}")
    my_answer = state["my_answer"]

    return {"answers_agent": [my_answer], "ord_aspects": [state["aspect"]]}


def grade_documents(state,verbose,llm):
    """
    Determines whether the retrieved documents are relevant to the question
    If any document is not relevant, we will set a flag to run web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """
    if verbose:
        print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
        print(f"State: {state}")
    query = state["query"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    web_search = "No"
    for d in documents:
        score = retrieval_grader(llm).invoke(
            {"question": query, "document": d.page_content}
        )
        #grade = score["score"]
        # Document relevant
        if score.lower() == "yes":
            if verbose: print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        # Document not relevant
        else:
            if verbose: print("---GRADE: DOCUMENT NOT RELEVANT---")
            # We do not include the document in filtered_docs
            # We set a flag to indicate that we want to run web search
            web_search = "Yes"
            continue
    return {"documents": filtered_docs, "query": query, "web_search": web_search}


def web_search(state,verbose):
    """
    Web search based based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """
    if verbose:
        print("---WEB SEARCH---")
        print(f"State: {state}")
    query = state["query"]
    documents = state["documents"]

    # Web search
    docs = web_search_tool.invoke({"query": query})
    #print(f"docs from web: {docs}") #Debug
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=250, chunk_overlap=0
    )

    doc_splits = text_splitter.split_documents([web_results])
    for doc in doc_splits:
        if documents is None:
            documents = [doc]
        else:
            documents.append(doc)
    return {"documents": documents, "query": query}


def hate_speech_filter(state,verbose,llm):
    if verbose:
        print("---HATE SPEECH FILTER---")
        print(f"State: {state}")
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = hate_speech_detection(llm).invoke(
            {"document": d.page_content}
        )
        #grade = score["score"]
        if score.lower() == "no":
            if verbose: print("---DOCUMENT ACCEPTED---")
            filtered_docs.append(d)
    
    return {"documents": filtered_docs}


def entailment_filter(state,BART_model,strategy_entailment,neutral_acceptance,verbose,test_mode,observer,llm):
    """
    Filter documents that doesn't entail with KBT

    Args:
        state (dict): The current graph state
    """
    
    if verbose:
        print("---ENTAILMENT FILTER---")
        print(f"State: {state}")
    query = state["query"]
    documents = state["documents"]
    documents_KBT = state["documents_kbt"]
    aspect_id = state["aspect_id"]

    # Score each doc
    filtered_docs = []
    #counter_docs = 0 # Debug
    if strategy_entailment: #Skeptical
        for d in documents:
            #counter_docs = counter_docs + 1 #Debug
            neutral = True
            for d_kbt in documents_KBT:
                if BART_model:
                    score = BART_prediction(d_kbt.page_content,d.page_content)
                else:
                    score = entailment_checker(llm).invoke(
                        {"first_doc": d_kbt.page_content, "second_doc": d.page_content}
                    )
                #grade = score["score"]
                
                #if score.lower() == "neutral": print(f"---Neutral {counter_docs}---") #Debug
                #if score.lower() == "entailment": print(f"---Entailment {counter_docs}---") #Debug
                #if score.lower() == "contradiction": print(f"---Contradiction {counter_docs}---") #Debug
                
                if score.lower() != "neutral":
                    neutral = False
                if score.lower() == "contradiction":
                    # contradiction found
                    break
            if (not neutral or neutral_acceptance) and score.lower() != "contradiction":
                filtered_docs.append(d)
                #print(f"---Document accepted {counter_docs}---")  #Debug            
                if verbose: print("---DOCUMENT ENTAILED---")   
    else: #Credolous
        for d in documents:
            #counter_docs = counter_docs + 1 #Debug
            neutral = True
            for d_kbt in documents_KBT:
                if BART_model:
                    score = BART_prediction(d_kbt.page_content,d.page_content)
                else:
                    score = entailment_checker(llm).invoke(
                        {"first_doc": d_kbt.page_content, "second_doc": d.page_content}
                    )
                #grade = score["score"]
                
                #if score.lower() == "neutral": print(f"---Neutral {counter_docs}---") #Debug
                #if score.lower() == "entailment": print(f"---Entailment {counter_docs}---") #Debug
                #if score.lower() == "contradiction": print(f"---Contradiction {counter_docs}---") #Debug
                
                # Document entailed
                if score.lower() != "neutral":
                    neutral = False
                if score.lower() == "entailment":
                    if verbose: print("---DOCUMENT ENTAILED---")
                    #print(f"---Document accepted {counter_docs}---")  #Debug
                    filtered_docs.append(d)
                    break
            if (neutral and neutral_acceptance) and score.lower() != "entailment":
                filtered_docs.append(d)
                #print(f"---Document accepted {counter_docs}---")  #Debug
                if verbose: print("---DOCUMENT ENTAILED---")
    
    if test_mode:
        #todo misura metriche con sklearn classifier
        #0 tweet veri, #1 tweet falsi
        y_true = [int(document.metadata.get("label")) for document in documents]
        y_pred = [0 if document in filtered_docs else 1 for document in documents]
        #print(f"Real docs with aspect {aspect_id} ,y_true: {y_true}") #Debug
        #print(f"Filtered docs with aspect {aspect_id} ,y_pred: {y_pred}") #Debug
        report = sklearn.metrics.classification_report(y_true,y_pred,labels=[0,1],
                                                       output_dict=True,zero_division=0)
        my_dict = {f"aspect_{aspect_id}": {"query": query, "report": report}}
        observer.generated_queries.update(my_dict)
    return {"documents": filtered_docs}


def debiasing(state,verbose,llm):
    if verbose:
        print("---DEBIASING FILTER---")
        print(f"State: {state}")
        
    answer = state["my_answer"]
    
    unbiased_answer = debiasing_answer(llm).invoke({"text": answer})
    
    return {"answers_agent": [unbiased_answer], "ord_aspects": [state["aspect"]]}


### Conditional edge

def bias_detection(state,verbose,llm,bias_encoder_model):
    if verbose:
        print("---BIAS DETECTION---")
        print(f"State: {state}")
        
    answer = state["my_answer"]
    response = ""
    
    if bias_encoder_model:
        bias_detection = pipeline('text-classification', model=bias_model, tokenizer=bias_model_tokenizer, device=device) # cuda = 0,1 based on gpu availability
        response = bias_detection(answer)[0]['label'].lower()
    else:
        response = bias_det_llm(llm).invoke({"text": answer})
        if ("biased" not in response) or ("non-biased" not in response):
            response = "biased"
        
    if verbose: print(response) #biased, non-biased
    
    return response

    
#Not used
def route_question(state,verbose,llm):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """
    if verbose:
        print("---ROUTE QUESTION---")
        print(f"State: {state}")
    query = state["query"]
    #print(queries)
    source = question_router(llm).invoke({"question": query})
    #print(source)
    #print(source["datasource"])
    if source["datasource"] == "web_search":
        if verbose: print("---ROUTE QUESTION TO WEB SEARCH---")
        return "websearch"
    elif source["datasource"] == "vectorstore":
        if verbose: print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"


def decide_to_generate(state,verbose):
    """
    Determines whether to generate an answer, or add web search

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """
    if verbose:
        print("---ASSESS GRADED DOCUMENTS---")
        print(f"State: {state}")
    web_search = state["web_search"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        if verbose: print(
                "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---"
            )
        return "websearch"
    else:
        # We have relevant documents, so generate answer
        if verbose: print("---DECISION: RELEVANT---")
        return "relevant"


# Not used
def grade_generation_v_documents_and_question(state,verbose,llm):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """
    
    if verbose:
        print("---CHECK HALLUCINATIONS---")
        print(f"State: {state}")
    query = state["query"]
    documents = state["documents"]
    my_answer = state["my_answer"]

    score = hallucination_grader(llm).invoke(
        {"documents": documents, "generation": my_answer} #answers_agent[0]
    )
    if verbose: print(f"score: {score}")
    grade = score["score"]

    # Check hallucination
    if grade == "yes":
        if verbose: print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        if verbose: print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": query, "generation": my_answer})
        grade = score["score"]
        if grade == "yes":
            if verbose: print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            if verbose: print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        if verbose: pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

**Building graph with edges**

In [89]:
# Workflow condizionale
def workflow_aspect_agent(configs):
    # Build graph
    workflow = StateGraph(GraphState)

    workflow.add_node("rewrite_query", functools.partial(rewrite_query, verbose=configs.verbose, 
                                                    llm=configs.local_llm))  # query rewriting
    workflow.add_node("retrieve", functools.partial(retrieve, verbose=configs.verbose, 
                                                    retriever=configs.retriever, retrievers_KBT=configs.retrievers_KBT))  # retrieve
    workflow.add_node("generate", functools.partial(generate, verbose=configs.verbose, 
                                                    llm=configs.local_llm, fairness=configs.fairness))  # generatae
    
    if configs.web_search:
        workflow.add_node("websearch", functools.partial(web_search, verbose=configs.verbose))  # web search
        workflow.add_node("grade_documents", functools.partial(grade_documents, verbose=configs.verbose, 
                                                               llm=configs.local_llm))  # grade documents
    if configs.safeness:
        workflow.add_node("hate_speech_filter", functools.partial(hate_speech_filter, verbose=configs.verbose, 
                                                                  llm=configs.local_llm))
    if configs.trustworthiness:
        workflow.add_node("entailment_filter", functools.partial(entailment_filter, BART_model=configs.BART_model, strategy_entailment=configs.strategy_entailment, 
                                                                 neutral_acceptance=configs.neutral_acceptance, verbose=configs.verbose, llm=configs.local_llm, 
                                                                 test_mode=configs.test_mode, observer=configs.observer))  # entailment
    if configs.fairness:  
        workflow.add_node("debiasing_filter", functools.partial(debiasing, verbose=configs.verbose, llm=configs.local_llm))
        workflow.add_node("confirm_answer", functools.partial(confirm_answer, verbose=configs.verbose))

    # Non applichiamo il routing
    """
    workflow.add_conditional_edges(
        START,
        route_question,
        {
            "websearch": "websearch", #se la risposta è websearch, allora vai al nodo websearch
            "vectorstore": "retrieve", #se la risposta è vectorstore, allora vai al nodo retrieve
        },
    )
    """
    workflow.add_edge(START, "rewrite_query")
    workflow.add_edge("rewrite_query", "retrieve")
    
    if configs.web_search:
        workflow.add_edge("retrieve", "grade_documents")  
        workflow.add_conditional_edges(
            "grade_documents",
            functools.partial(decide_to_generate, verbose=configs.verbose),
            {
                "websearch": "websearch",
                "relevant": "hate_speech_filter" if configs.safeness else "entailment_filter" if configs.trustworthiness else "generate"
            },
        )
    
    if configs.safeness:
        if configs.web_search:
            workflow.add_edge("websearch", "hate_speech_filter")
        else: 
            workflow.add_edge("retrieve", "hate_speech_filter")
        if configs.trustworthiness:
            workflow.add_edge("hate_speech_filter", "entailment_filter")
            workflow.add_edge("entailment_filter", "generate")
        else:
            workflow.add_edge("hate_speech_filter","generate")
    elif configs.trustworthiness:
        if configs.web_search:
            workflow.add_edge("websearch", "entailment_filter")
        else: 
            workflow.add_edge("retrieve", "entailment_filter")
        workflow.add_edge("entailment_filter", "generate")
    else:
        if configs.web_search:
            workflow.add_edge("websearch", "generate")
        else: 
            workflow.add_edge("retrieve", "generate")
    
    if configs.fairness:
        workflow.add_conditional_edges(
            "generate",
            functools.partial(bias_detection, verbose=configs.verbose, llm=configs.local_llm, bias_encoder_model=configs.bias_encoder_model),
            {
                "biased": "debiasing_filter",
                "non-biased": "confirm_answer",
            },
        )
        workflow.add_edge("confirm_answer", END)
        workflow.add_edge("debiasing_filter", END)
    else:
        workflow.add_edge("generate", END)

    # Non faccio il controllo sulle allucinazioni
    """
    workflow.add_conditional_edges(
        "generate",
        grade_generation_v_documents_and_question,
        {
            "not supported": "generate",
            "useful": END,
            "not useful": "websearch",
        },
    )
    """    
    workflow_compiled = workflow.compile()
    return workflow_compiled

In [ ]:
from IPython.display import Image, display

display(Image(workflow_aspect_agent(configs).get_graph().draw_mermaid_png()))

# Master agent

In [96]:
from typing import Annotated
import operator
from langgraph.constants import Send


### Super Graph State
class SuperGraphState(TypedDict):
    """
    Represents the state of our super-graph.
    """
    
    question: str
    aspects: List[str]
    answers_agent: Annotated[List[str], operator.add]
    ord_aspects: Annotated[List[str], operator.add]
    final_answer: str

def send_aspects(state,verbose):
    if verbose: 
        print("---SEND ASPECT TO EACH ASPECT-AGENT---")
        print(f"State: {state}")
    return [Send("aspect_agent_node", {"original_query": state["question"], "aspect": a, "aspect_id": state["aspects"].index(a)}) for a in state["aspects"]]

def organize_answers(state,verbose,llm,organize):
    """
    Organize the outputs of the agents.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, answer, that contains the final answer to give to user
    """
    if verbose:
        print("---ORGANIZE OUTPUTS---")
        print(f"State: {state}")
    answers_agent = state["answers_agent"]
    ord_aspects = state["ord_aspects"]
    
    #print(f"I'm the master agent and I received: {answers_agent}") #Debug
    if organize:
        final_output=final_answer_section(llm).invoke({"answers": answers_agent, "aspects": ord_aspects})
    else:
        final_output=final_answer(llm).invoke({"answers": answers_agent})
    return {"final_answer": final_output}

In [93]:
def master_flow(configs):
    master_flow = StateGraph(SuperGraphState)

    # Define the nodes
    master_flow.add_node("organize_answers", functools.partial(organize_answers, verbose=configs.verbose, llm=configs.local_llm, organize=configs.organize))
    master_flow.add_node("aspect_agent_node",workflow_aspect_agent(configs))

    # Build graph
    master_flow.add_conditional_edges(START, functools.partial(send_aspects, verbose=configs.verbose), ["aspect_agent_node"])
    master_flow.add_edge("aspect_agent_node", "organize_answers")
    master_flow.add_edge("organize_answers", END)

    master_compiled = master_flow.compile()
    return master_compiled

In [ ]:
from IPython.display import Image, display

# Setting xray to 1 will show the internal structure of the nested graph
display(Image(master_flow(configs).get_graph().draw_mermaid_png()))

# Configuration and app-launching

Vectorstore configuration

In [ ]:
index_name = "entailment-test"
aspects = ["Health","Technology","Society"] #"Technology", "Society"

top_retriever = 10 #documents retrieved by retriever
top_KBT = 5 #documents retrievede by KBT retriever

retriever = create_retriever(index_name, top_retriever)
retrievers_KBT = create_KBT_retrievers(aspects, top_KBT)

In [94]:
class Config(object):
    def __init__(self,retriever,retrievers_KBT,aspects):
        self.local_llm = "llama3.1" #"llama3.1:70b"
        
        # retrievers
        self.retriever = retriever
        self.retrievers_KBT = retrievers_KBT
        self.aspects = aspects
        
        #if we want print all the process: True
        self.verbose = False 
        
        #if we want to include websearch in the workflow
        self.web_search = True

        # Controlling properties
        self.safeness = True # if we want to add hate speech detection module
        self.trustworthiness = True # if we want to add entailment module with KBT
        self.fairness = True  # if we want to add debiasing module.

        #Controlling entailment
        # strategy for the entailment, False = "Credolous", True = "Skeptical" 
        self.strategy_entailment = True
        #manage the total neutral entailed documents (what if a document is neutral with all documents of KBT)
        # True = accept the neutral documents, False = don't accept
        self.neutral_acceptance = True   
        # True: uses BART model for the entailment, False: uses LLM
        self.BART_model = False
        
        #controlling bias detection
        # True: uses encoder model. False: uses LLM
        self.bias_encoder_model = True
        
        #controlling final generation 
        # True: organize final output in section. False: organize output without sections
        self.organize = True
        
        # For testing
        self.test_mode = False
        self.observer = None
    
configs = Config(retriever,retrievers_KBT,aspects)

Starting language model

In [ ]:
start_ollama()
pull_model(configs.local_llm)
start_model(configs.local_llm)

In [97]:
inputs = {"question": "Covid19 was a hoax?", "aspects": configs.aspects}

for output in master_flow(configs).stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
answer = value["final_answer"]
print(answer)

[GIN] 2024/08/13 - 14:14:04 | 200 |   1.21611314s |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/08/13 - 14:14:04 | 200 |  1.256974172s |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/08/13 - 14:14:04 | 200 |  1.379350696s |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/08/13 - 14:14:07 | 200 |  343.292979ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/08/13 - 14:14:07 | 200 |  318.997993ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/08/13 - 14:14:07 | 200 |  720.907976ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/08/13 - 14:14:07 | 200 |  546.493012ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/08/13 - 14:14:07 | 200 |  572.036761ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/08/13 - 14:14:08 | 200 |  373.047636ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/08/13 - 14:14:08 | 200 |  374.521972ms |       127.0.0.1 | POST     "/api/chat"
[GIN] 2024/08/13 - 14:14:08 | 200 |   342.35609ms |       127.0.0.1 | POST     "/api/chat"

# Test

In [ ]:
# Funzioni di supporto:
import json

def load_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

def write_file(filename,content):
    with open(filename, 'w') as file:
        json.dump(content, file, indent=4)
    
queries_gpt = ["Does the COVID-19 vaccine contain a microchip for controlling people?",
"Was the COVID-19 pandemic planned by a global elite?",
"Does inhaling hot steam kill the COVID-19 virus?",
"Was COVID-19 created in a lab as a biological weapon?",
"Does drinking alcohol help prevent COVID-19 infection?",
"Is 5G responsible for spreading COVID-19?",
"Do masks cause a reduction in blood oxygen levels?",
"Do COVID-19 vaccines cause infertility in women?",
"Can high doses of vitamin C cure COVID-19?",
"Can pets transmit COVID-19 to humans?"]

#write_file("/kaggle/working/gpt_queries.json",queries_gpt)

Predisposizione dell'observer per salvare i risultati

In [ ]:
class Observer(object):
    def __init__(self):
        self.query=""
        self.type_of_acceptance=""
        self.neutral_acceptance=False
        self.generated_queries={}
        self.final_answer=""
    
    def generate_dict(self):
        return {"query": self.query,
               "type_of_acceptance": self.type_of_acceptance,
               "neutral_acceptance": self.neutral_acceptance,
               "generated_queries": self.generated_queries,
               "final_answer": self.final_answer}

Combinazioni di configurazione con skeptical/credulous e neutral

Multiple combination:

In [ ]:
combination = [(True,True), (True,False), (False,True),(False,False)] #Skeptical-Neutral #Skeptical-No-Neutral #Cred-Neu #Cred-No-Neu

for comb in combination:
    print("Start combination")
    configs.strategy_entailment =  comb[0]
    configs.neutral_acceptance = comb[1]
    
    queries_list = queries_gpt

    attempt = 1
    ret_dict = {}
    for query in queries_list:
        inputs = {"question": query, "aspects": configs.aspects}
        print(f"Attempt {attempt} start")
    
        configs.observer = Observer()
        configs.observer.query=query
        if configs.strategy_entailment:
            configs.observer.type_of_acceptance="Skeptical"
        else:
            configs.observer.type_of_acceptance="Credulous"
        configs.observer.neutral_acceptance=configs.neutral_acceptance

        for output in master_flow(configs).stream(inputs):
            for key, value in output.items():
                pass
                #pprint(f"Finished running: {key}:")
        answer = value["final_answer"]
    
        configs.observer.final_answer= answer
        ret_dict[f"attempt {attempt}"] = configs.observer.generate_dict()
    
        attempt = attempt + 1

    stringa = "Neutral" if configs.neutral_acceptance else "No-Neutral"
    write_file(f"/kaggle/working/test_llama31_third_{configs.observer.type_of_acceptance}_{stringa}.json",ret_dict)

Single combination:

In [ ]:
#queries_list = load_queries("/kaggle/input/preference/gpt_queries.json")
queries_list = queries_gpt

attempt = 1
ret_dict = {}
for query in queries_list:
    inputs = {"question": query, "aspects": configs.aspects}
    print(f"Attempt {attempt} start")
    
    configs.observer = Observer()
    configs.observer.query=query
    if configs.strategy_entailment:
        configs.observer.type_of_acceptance="Skeptical"
    else:
        configs.observer.type_of_acceptance="Credulous"
    configs.observer.neutral_acceptance=configs.neutral_acceptance

    for output in master_flow(configs).stream(inputs):
        for key, value in output.items():
            pass
            #pprint(f"Finished running: {key}:")
    answer = value["final_answer"]
    
    configs.observer.final_answer= answer
    ret_dict[f"attempt {attempt}"] = configs.observer.generate_dict()
    
    attempt = attempt + 1

stringa = "Neutral" if configs.neutral_acceptance else "No-Neutral"
write_file(f"/kaggle/working/test_llama31_{configs.observer.type_of_acceptance}_{stringa}.json",ret_dict)

In [ ]:
print(ret_dict)

# Evaluation

## Entailment evaluation

In [13]:
import pandas as pd

### Valutazione su notizie vere (0) e notizie false (1)

In [20]:
# Label 0, notizie vere
# Label 1, notizie false

def results_on_label(label, num_attempts, num_aspects, dict_input):
    recall = 0
    precision = 0
    f1 = 0
    support = 0

    for i in range(1,num_attempts+1):
        for j in range(num_aspects):
            recall = recall + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{j}"]["report"][label]["recall"]
            precision = precision + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{j}"]["report"][label]["precision"]
            f1 = f1 + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{j}"]["report"][label]["f1-score"]
            support = support + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{j}"]["report"][label]["support"]

    return {"recall":recall/(num_attempts*num_aspects), 
            "precision":precision/(num_attempts*num_aspects),
            "f1":f1/(num_attempts*num_aspects), 
            "support":support/(num_attempts*num_aspects)}

### Valutazione per aspetto

In [31]:
def results_on_aspects(aspect_id, num_attempts, num_aspects, dict_input):
    recall = 0
    precision = 0
    f1 = 0

    weight_recall = 0
    weight_precision = 0
    weight_f1 = 0
    accuracy = 0

    for i in range(1,num_attempts+1):
        recall = recall + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{aspect_id}"]["report"]["macro avg"]["recall"]
        precision = precision + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{aspect_id}"]["report"]["macro avg"]["precision"]
        f1 = f1 + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{aspect_id}"]["report"]["macro avg"]["f1-score"]
        weight_recall = weight_recall + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{aspect_id}"]["report"]["weighted avg"]["recall"]
        weight_precision = weight_precision + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{aspect_id}"]["report"]["weighted avg"]["precision"]
        weight_f1 = weight_f1 + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{aspect_id}"]["report"]["weighted avg"]["f1-score"]
        try:
            accuracy = accuracy + dict_input[f"attempt {i}"]["generated_queries"][f"aspect_{aspect_id}"]["report"]["accuracy"]
        except:
            accuracy = accuracy + 1 #se l'accuracy non è presente nel dizionario, è perché vale 1 (vedi esempio 4 di llama_second_skeptical_neutral)
    
    return {"recall": recall/(num_attempts),
            "precision": precision/(num_attempts), 
            "f1":f1/(num_attempts),
            "w_recall": weight_recall/(num_attempts),
            "w_precision": weight_precision/(num_attempts), 
            "w_f1":weight_f1/(num_attempts), 
            "accuracy":accuracy/(num_attempts)}

In [22]:
import json

def load_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

def write_file(filename,content):
    with open(filename, 'w') as file:
        json.dump(content, file, indent=4)

In [26]:
# label 0: notizie vere (negative class)
# label 1: notizie false (positive class)

def compute_result_on_label(model, shot, acceptance, neutral, attempts, num_aspects) -> pd.DataFrame:
    if shot=="first" or model=="BART":
        dict_input = load_json(f"/kaggle/working/test_{model}_{acceptance}_{neutral}.json")
    else:
        dict_input = load_json(f"/kaggle/working/test_{model}_{shot}_{acceptance}_{neutral}.json")

    results_label_0 = results_on_label("0", attempts, num_aspects, dict_input)
    results_label_1 = results_on_label("1", attempts, num_aspects, dict_input)
    

    df = pd.DataFrame()
    df["model"] = [model]
    df["shots"] = [shot]
    df["type_acceptance"] = [acceptance]
    df["neutral"] = [neutral]

    df["precision_0"] = results_label_0["precision"]
    df["recall_0"] = results_label_0["recall"]
    df["f1_0"] = results_label_0["f1"]
    df["support_0"] = results_label_0["support"]
    

    df["precision_1"] = results_label_1["precision"]
    df["recall_1"] = results_label_1["recall"]
    df["f1_1"] = results_label_1["f1"]
    df["support_1"] = results_label_1["support"]

    return df

In [27]:
def compute_result_on_aspects(model, shot, acceptance, neutral, attempts, num_aspects, aspects) -> pd.DataFrame:
    if shot=="first" or model=="BART":
        dict_input = load_json(f"/kaggle/working/test_{model}_{acceptance}_{neutral}.json")
    else:
        dict_input = load_json(f"/kaggle/working/test_{model}_{shot}_{acceptance}_{neutral}.json")

    df = pd.DataFrame()
    df["model"] = [model]
    df["shots"] = [shot]
    df["type_acceptance"] = [acceptance]
    df["neutral"] = [neutral]
    
    aspect_id = 0
    for aspect in aspects:
        result = results_on_aspects(aspect_id, attempts, num_aspects, dict_input)
        df[f"precision_{aspect}"] = result["precision"]
        df[f"recall_{aspect}"] = result["recall"]
        df[f"f1_{aspect}"] = result["f1"]
        
        df[f"accuracy_{aspect}"] = result["accuracy"]
        df[f"w_precision_{aspect}"] = result["w_precision"]
        df[f"w_recall_{aspect}"] = result["w_recall"]
        df[f"w_f1_{aspect}"] = result["w_f1"]
        
        aspect_id = aspect_id+1    

    return df

In [29]:
import openpyxl

attempts = 10
num_aspects = 3

models = ["BART","llama31"]
shots = ["first","second","third"]
type_acceptance = ["Skeptical","Credulous"]
neutral_acceptance = ["No-Neutral","Neutral"]
aspects = ["Health","Technology","Society"]

excel_file = "/kaggle/working/test_entailment_results_labels.xlsx"
results = pd.DataFrame()
for model in models:
    for shot in shots:
        for acceptance in type_acceptance:
            for neutral in neutral_acceptance:
                new_row = compute_result_on_label(model, shot, acceptance, neutral, attempts, num_aspects)
                results = pd.concat([results, new_row], ignore_index=True)
        if model=="BART": break

print(results.round(3))
results.round(3).to_excel(excel_file, index=False, engine='openpyxl')

      model   shots type_acceptance     neutral  precision_0  recall_0   f1_0  \
0      BART   first       Skeptical  No-Neutral        0.133     0.052  0.065   
1      BART   first       Skeptical     Neutral        0.416     0.461  0.395   
2      BART   first       Credulous  No-Neutral        0.200     0.080  0.103   
3      BART   first       Credulous     Neutral        0.408     0.453  0.377   
4   llama31   first       Skeptical  No-Neutral        0.547     0.467  0.472   
5   llama31   first       Skeptical     Neutral        0.546     0.707  0.577   
6   llama31   first       Credulous  No-Neutral        0.459     0.465  0.378   
7   llama31   first       Credulous     Neutral        0.567     0.760  0.598   
8   llama31  second       Skeptical  No-Neutral        0.482     0.461  0.452   
9   llama31  second       Skeptical     Neutral        0.585     0.739  0.618   
10  llama31  second       Credulous  No-Neutral        0.517     0.524  0.464   
11  llama31  second       Cr

In [32]:
excel_file = "/kaggle/working/test_entailment_results_aspects.xlsx"
results = pd.DataFrame()
for model in models:
    for shot in shots:
        for acceptance in type_acceptance:
            for neutral in neutral_acceptance:
                new_row = compute_result_on_aspects(model, shot, acceptance, neutral, attempts, num_aspects, aspects)
                results = pd.concat([results, new_row], ignore_index=True)
        if model=="BART": break

print(results.round(3))
results.round(3).to_excel(excel_file, index=False, engine='openpyxl')

      model   shots type_acceptance     neutral  precision_Health  \
0      BART   first       Skeptical  No-Neutral             0.441   
1      BART   first       Skeptical     Neutral             0.627   
2      BART   first       Credulous  No-Neutral             0.504   
3      BART   first       Credulous     Neutral             0.627   
4   llama31   first       Skeptical  No-Neutral             0.714   
5   llama31   first       Skeptical     Neutral             0.737   
6   llama31   first       Credulous  No-Neutral             0.641   
7   llama31   first       Credulous     Neutral             0.670   
8   llama31  second       Skeptical  No-Neutral             0.765   
9   llama31  second       Skeptical     Neutral             0.787   
10  llama31  second       Credulous  No-Neutral             0.756   
11  llama31  second       Credulous     Neutral             0.765   
12  llama31   third       Skeptical  No-Neutral             0.731   
13  llama31   third       Skeptica

# Ragas (application evaluation)

https://docs.ragas.io/en/stable/

In [ ]:
#todo